# Obtaining the best redshift from 3 Catalogs: Griffiths, 3dHST, UltraVISTA

## What we have:
## Griffiths has spec-z, photo-z. Their best Z is 'high-quality spec z else photoz.' It is not known how high-quality is defined. The errors listed on their photoz are 3sig. 
## 3dHST has spec-zs, photo-zs, and grism-zs. Their 'z_best' can be any of these.
## UltraVISTA just has photo-zs. 

## Workflow for choosing best redshift:
## 1) If Griffiths spec-z = Griffiths best-z, then use that. 
## else if 3dHST spec-z = 3dHST best_z, use that. 
## else (no good spec-z's anywhere):
## compare 1-sig errors on Griffiths photoz, 3dHST grism z, 3dHST photoz, UltraVISTA photoz. choose that with lowest err. Note: all except Griffiths report 1 sig errors; Griffiths reports 3sig, so that error is divided by 3 in the comparison.

 



In [23]:
%matplotlib inline

from astropy.io import fits as pyfits
from astropy.table import Table
import numpy as np
from astropy.io.fits import Column
from datetime import datetime
from operator import itemgetter
import matplotlib.pyplot as plt
import os
import warnings
import requests

In [5]:
def download_from_dropbox(url):
    
    local_filename = "{:}".format(url.split("/")[-1].split("?")[0])
    r = requests.get(url, stream=True)
    with open(local_filename, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024): 
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                f.flush()
            
    return local_filename

In [19]:
old_catalog_filename = download_from_dropbox("https://www.dropbox.com/s/ge7sgtf0crxmwyi/all_redshift_info.fits?dl=1")
old_catalog = Table.read(old_catalog_filename)


In [7]:
subjects=set(old_catalog['OBJNO'])
imagings=set(old_catalog['IMAGING'])

In [20]:
updated_table = old_catalog.copy(copy_data=True)

In [15]:
imagings

{'AEGIS     ',
 'COSMOS    ',
 'GEMS      ',
 'GOODS-N   ',
 'GOODS-S   ',
 'GOODS_FULL',
 'SDSS      '}

In [21]:
def get_best_redshift(galaxy):
    
    #define Griffiths variables
    griffiths_specz = galaxy['SPECZ']
    griffiths_bestz = galaxy['Z']
    griffiths_photoz = galaxy['PHOTOZ']
    griffiths_photoz_err_3sig = galaxy['PHOTOZ_ERR']/galaxy['PHOTOZ']
    griffiths_photoz_err_1sig = galaxy['PHOTOZ_ERR']/galaxy['PHOTOZ']/3.

    
    #define 3dhst variables
    dhst_specz = galaxy['z_spec']
    dhst_bestz = galaxy['z_best']
    dhst_photoz = galaxy['z_peak_phot']
    dhst_photoz_err = (galaxy['z_phot_u68']-galaxy['z_phot_l68'])/dhst_photoz

    dhst_grismz = galaxy['z_max_grism']
    dhst_grismz_err = (galaxy['z_grism_u68']-galaxy['z_grism_l68'])/dhst_grismz

    
    if str(dhst_photoz_err)=='nan':
        dhst_photoz_err = 99
    if dhst_photoz < 0:
        dhst_photoz_err = 99
    if str(dhst_grismz_err)=='nan':
        dhst_grismz_err = 99
    if dhst_grismz < 0:
        dhst_grismz_err = 99
    
    #define ultravista variables
    uv_photoz = galaxy['zPDF']
    uv_photoz_err = (galaxy['zPDF_u68']-galaxy['zPDF_l68'])/uv_photoz
    if str(uv_photoz_err)=='nan':
        uv_photoz_err = 99
    if uv_photoz < 0:
        uv_photoz_err=99
    
    #Order of operations to determine which redshift to use:
    #1) If specz exists in Griffiths catalog, use that first. 
    if griffiths_specz == griffiths_bestz and griffiths_specz > 0 and griffiths_specz < 9:
        return griffiths_specz,'Griffiths_specz'
        
    #if no specz in Griffiths, check 3Dhst
    else:
        if dhst_specz == dhst_bestz and dhst_specz >0 and dhst_specz <9: 
            return dhst_bestz, '3DHST_specz'
        
        else: #no speczs, so check which photoz has smallest error. 
            photoz_list = [[griffiths_photoz,griffiths_photoz_err_1sig,'Griffiths_photoz'],[dhst_photoz,dhst_photoz_err,'3DHST_photoz'],[uv_photoz,uv_photoz_err,'UltraVISTA_photoz'],[dhst_grismz,dhst_grismz_err,'3DHST_grismz']]
            photoz_list = sorted(photoz_list, key=itemgetter(1)) #sort by which has smallest error            
            
            return photoz_list[0][0],photoz_list[0][2]

In [24]:
for galaxy in updated_table:
    if galaxy['IMAGING'].strip()!='SDSS' and galaxy['IMAGING'].strip()!='GOODS_FULL':
        galaxy['Z_BEST'], galaxy['Z_BEST_SOURCE'] = get_best_redshift(galaxy)

/home/user1/galloway/anaconda2/envs/Zuko/lib/python2.7/site-packages/ipykernel/__main__.py:7: RuntimeWarning: divide by zero encountered in float_scalars
/home/user1/galloway/anaconda2/envs/Zuko/lib/python2.7/site-packages/ipykernel/__main__.py:8: RuntimeWarning: divide by zero encountered in float_scalars
/home/user1/galloway/anaconda2/envs/Zuko/lib/python2.7/site-packages/ipykernel/__main__.py:7: RuntimeWarning: invalid value encountered in float_scalars
/home/user1/galloway/anaconda2/envs/Zuko/lib/python2.7/site-packages/ipykernel/__main__.py:8: RuntimeWarning: invalid value encountered in float_scalars


In [25]:
fname = '/home/user1/galloway/Dropbox/gzhubble/hubble_files/gzh_redshift_data/best_redshift_table_%i_%i_%i.fits'%(datetime.now().month,datetime.now().day,datetime.now().year)

if os.path.exists(fname):
    os.remove(fname)
updated_table.write(fname,format='fits')


In [40]:
#Check distribution of different redshifts
imaging_list=list(set(updated_table['IMAGING']))
imaging_list.remove(imaging_list[2])
imaging_list.remove(imaging_list[2])

z_sources = list(set(updated_table['Z_BEST_SOURCE']))
z_sources.remove(z_sources[0])

for survey in imaging_list:
    print('for survey %s:' %survey)
    for source in z_sources: 
        print ('%s:  = %i' %(source, ((updated_table['IMAGING'] == survey) & (updated_table['Z_BEST_SOURCE'] == source)).sum()))
    print('')
for source in z_sources:
    print('total %s = %i'%(source,((updated_table['Z_BEST_SOURCE']==source)).sum()))

for survey AEGIS     :
Griffiths_specz:  = 3656
3DHST_specz:  = 12
Griffiths_photo:  = 4442
3DHST_grismz:  = 341
UltraVISTA_phot:  = 0
3DHST_photoz:  = 56

for survey GOODS-N   :
Griffiths_specz:  = 1997
3DHST_specz:  = 5
Griffiths_photo:  = 218
3DHST_grismz:  = 101
UltraVISTA_phot:  = 0
3DHST_photoz:  = 230

for survey GEMS      :
Griffiths_specz:  = 387
3DHST_specz:  = 6
Griffiths_photo:  = 8886
3DHST_grismz:  = 9
UltraVISTA_phot:  = 0
3DHST_photoz:  = 16

for survey GOODS-S   :
Griffiths_specz:  = 3016
3DHST_specz:  = 103
Griffiths_photo:  = 1662
3DHST_grismz:  = 67
UltraVISTA_phot:  = 0
3DHST_photoz:  = 65

for survey COSMOS    :
Griffiths_specz:  = 7201
3DHST_specz:  = 35
Griffiths_photo:  = 82662
3DHST_grismz:  = 358
UltraVISTA_phot:  = 2473
3DHST_photoz:  = 79

total Griffiths_specz = 16257
total 3DHST_specz = 161
total Griffiths_photo = 97870
total 3DHST_grismz = 876
total UltraVISTA_phot = 2473
total 3DHST_photoz = 446


In [36]:
z_sources



{'               ',
 '3DHST_grismz',
 '3DHST_photoz',
 '3DHST_specz',
 'Griffiths_photo',
 'Griffiths_specz',
 'UltraVISTA_phot'}

In [33]:
imaging_list

['AEGIS     ', 'GOODS-N   ', 'GEMS      ', 'GOODS-S   ', 'COSMOS    ']